# Pydantic

Pydantic to biblioteka służąca do tworzenia i walidacji modeli struktur danych czyli schematów danych o strukturze typu JSON.

Przykładowe **zastosowania** modeli pydantica:
- definiowanie request body w REST API
- definiowanie formatu formatu JSONa według którego odpowiedź ma wygenerować LLM
- walidacja poprawności formatu danych JSON
- zamiana danych w postaci słownika na instancję klasy modelu (możemy wtedy np. zdefiniować dla takich obiektów odpowiednie metody)

## `BaseModel` i modele danych

In [1]:
from pydantic import BaseModel

In [3]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool

In [4]:
class Task(BaseModel):
    description: str
    priority: int
    is_completed: bool
    assigned_user: User

## Walidacja JSONa

Do walidacji JSONa względem modelu używamy metody `model_validate()`.

**Poprawna walidacja**

W przypadku kiedy słownik pasuje do modelu, `model_validate()` zwraca instancję tego modelu.

In [5]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}

task = {"description": "Task description", "priority": 2, "is_completed": False, "assigned_user": user}

In [6]:
User.model_validate(user)

User(username='John Doe', password='my_password', is_admin=True)

In [7]:
Task.model_validate(task)

Task(description='Task description', priority=2, is_completed=False, assigned_user=User(username='John Doe', password='my_password', is_admin=True))

---

Poniżej `priority` jest zapisane jako `str`. Ponieważ jednak da się to zrzutować na `int`, to walidacja również przebiega poprawnie.

In [10]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}
task = {"description": "Task description", "priority": "abc", "is_completed": False, "assigned_user": user}

In [11]:
Task.model_validate(task)

ValidationError: 1 validation error for Task
priority
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

**Niepoprawna walidacja**

W przypadku błędu walidacji dostajemy błąd `ValidationError`.

In [12]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}
task = {"description": "Task description", "priority": "invalid type", "is_completed": False, "assigned_user": user}

In [13]:
Task.model_validate(task)

ValidationError: 1 validation error for Task
priority
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid type', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

## Rzutowanie słownika na obiekt

Rzutowanie słownika na obiekt pozwala nie tylko przechowywać informacje w postaci tego obiektu zamiast słownika ale również korzystać ze wszystkich metod odpowiadającej mu klasy.

In [16]:
user

{'username': 'John Doe', 'password': 'my_password', 'is_admin': True}

In [17]:
# User(username=user["username"])

In [15]:
User(**user)

User(username='John Doe', password='my_password', is_admin=True)

In [18]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool

    def change_password(self, new_password):
        if isinstance(new_password, str):
            self.password = new_password
        else:
            raise TypeError("Invalid type for new password")

In [19]:
user_obj = User(**user)
user_obj

User(username='John Doe', password='my_password', is_admin=True)

In [20]:
user_obj.change_password("new_password")
user_obj

User(username='John Doe', password='new_password', is_admin=True)

In [22]:
user_obj.change_password("True")

In [23]:
user_obj

User(username='John Doe', password='True', is_admin=True)

In [24]:
user_obj.is_admin = False

In [25]:
user_obj

User(username='John Doe', password='True', is_admin=False)

> **ZADANIE**

Stwórz klasę `Task`, która będzie miała następujące pola (atrybuty). Dobierz dla nich odpowiednie typy.
   - `description`
   - `assigned_user`
   - `due_date`
   - `comments`

Klasa ta powinna również posiadać metodę `modify_description()`, której zadaniem będzie modyfikacja opisu zadania. Jeśli podano wartość typu innego niż `str` należy zwrócić odpowiedni błąd.

Następnie na podstawie słownika z danymi o zadaniu dokonaj walidacji i zamiany słownika na obiekt.

In [27]:
from datetime import date

In [28]:
class Task(BaseModel):
    description: str
    assigned_user: User
    due_date: date
    comments: list[str]

    def modify_description(self, value: str):
        if not isinstance(value, str):
            raise TypeError("Description must be a string")
        self.description = value

In [29]:
user

{'username': 'John Doe', 'password': 'my_password', 'is_admin': True}

In [34]:
task_dict = {"description": "Opis", "assigned_user": user, "due_date": date.today(), "comments": []}

task_obj = Task(**task_dict)
task_obj

Task(description='Opis', assigned_user=User(username='John Doe', password='my_password', is_admin=True), due_date=datetime.date(2025, 11, 21), comments=[])

In [35]:
Task.model_validate(task_dict)

Task(description='Opis', assigned_user=User(username='John Doe', password='my_password', is_admin=True), due_date=datetime.date(2025, 11, 21), comments=[])

In [37]:
task_obj.modify_description("modified description")
task_obj

Task(description='modified description', assigned_user=User(username='John Doe', password='my_password', is_admin=True), due_date=datetime.date(2025, 11, 21), comments=[])

---
---
---

## Zaawansowane elementy definiowania modeli

**Dopuszczanie różnych typów**

In [38]:
class Task(BaseModel):
    description: str
    priority: int | str
    is_completed: bool | None

In [40]:
Task(description="task description", priority=2, is_completed=None)

Task(description='task description', priority=2, is_completed=None)

---

In [43]:
class Task(BaseModel):
    description: str
    priority: float | int
    is_completed: bool | None

In [44]:
Task(description="task description", priority=2, is_completed=False)

Task(description='task description', priority=2, is_completed=False)

**Typ `Literal`**

In [45]:
from typing import Literal

In [47]:
class Task(BaseModel):
    description: str
    priority: Literal["low", "medium", "high"]
    is_completed: bool

In [50]:
Task(description="task description", priority="medium123", is_completed=True)

ValidationError: 1 validation error for Task
priority
  Input should be 'low', 'medium' or 'high' [type=literal_error, input_value='medium123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error

**Typy z ograniczeniami**

Constrained types

In [51]:
from pydantic import constr, conint, confloat

In [52]:
class Test(BaseModel):
    test_string: constr(min_length=5, max_length=10, pattern=r"^[a-zA-Z]+$")
    test_int: conint(gt=2, lt=8)  # greater than, less then
    test_float: confloat(ge=0.0, le=1.0)

In [53]:
test = Test(test_string="hello", test_int=3, test_float=0.5)
test

Test(test_string='hello', test_int=3, test_float=0.5)

In [58]:
Test(test_string="hello", test_int=3, test_float=0.5)

Test(test_string='hello', test_int=3, test_float=0.5)

**Wartości domyślne i opcjonalne**

In [59]:
from datetime import date

In [60]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool = False
    date_of_birth: date | None = None

In [61]:
User(username="John Doe", password="password")

User(username='John Doe', password='password', is_admin=False, date_of_birth=None)

**Field – metadane pola**

`Field` pozwala zdefiniować wiele rzeczy w jednym miejscu, m.in. wartość domyślną, walidację wartości, alias pola czy jego opis.

In [63]:
from pydantic import Field
from datetime import datetime

In [68]:
class User(BaseModel):
    username: str = Field(alias="login")
    password: str = Field(min_length=8)
    is_admin: bool = Field(False, description="Whether or not the user is an admin")
    creation_timestamp: date = Field(default_factory=datetime.now)

In [69]:
User(login="my_username", password="my_password")

User(username='my_username', password='my_password', is_admin=False, creation_timestamp=datetime.datetime(2025, 11, 21, 14, 24, 42, 348842))

**`field_validator` i `model_validator` – customowa walidacja**

Jeśli chcemy zastosować bardziej zaawansowaną walidację modelu możemy użyć metod z dekoratorami `field_validator` lub `model_validator`.

`field_validator` - służy do walidacji wartości w pojedynczym polu

`model_validator` - służy do walidacji zależności między polami modelu

In [70]:
from pydantic import field_validator, model_validator

In [ ]:
from pydantic import ConfigDict

In [81]:
class User(BaseModel):
    # model_config = {"validate_assignment": True}
    
    username: str = Field(alias="login")
    password: str = Field(min_length=8)
    is_admin: bool = Field(False, description="Whether or not the user is an admin")
    creation_timestamp: date = Field(default_factory=datetime.now)

    @field_validator("password")
    def check_password_not_contains_admin(cls, value):
        if "admin" in value.lower():
            raise ValueError("Password should not contain the word 'admin'")
        return value

    @model_validator(mode="after")
    def validate_password_length_for_admins(self):
        min_admin_password_length = 12
        if self.is_admin and len(self.password) < min_admin_password_length:
            raise ValueError("Too short password for an admin")
        return self

In [83]:
u.password = "abc"

In [82]:
u = User(login="user", password="sfsd12342423424")

In [73]:
User(login="user", password="admin1234")  # admin1234

ValidationError: 1 validation error for User
password
  Value error, Password should not contain the word 'admin' [type=value_error, input_value='admin1234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [75]:
User(login="user", password="user1234567", is_admin=True)  # user1234

ValidationError: 1 validation error for User
  Value error, Too short password for an admin [type=value_error, input_value={'login': 'user', 'passwo...4567', 'is_admin': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

> **ZADANIE**

Rozbuduj klasę `Task` z poprzedniego zadania w następujący sposób:
- w polu `assigned_user` zezwól na wartość typu `str` (username użytkownika)
- dodaj pole `category`, które będzie miało kilka określonych wartości dopuszczalnych
- ustaw maksymalną długość opisu na 30 znaków
- przyjmij pustą listę jako domyślną wartość pola `comments`
- dodaj opis wybranego pola korzystając z `Field`

Następnie stwórz instancję tej klasy.

In [84]:
class Task(BaseModel):
    description: str = Field(max_length=30)
    assigned_user: User | str
    due_date: date
    comments: list[str] = []
    category: Literal["bug", "feature", "general"] = Field(description="Task category")

    def modify_description(self, value: str):
        if not isinstance(value, str):
            raise TypeError("Description must be a string")
        self.description = value

## Pydantic + LLM

Poniższego kodu **nie należy** wykonywać (jest do tego potrzebny klucz API OpenAI). Obrazuje on jednak w jaki sposób możemy zastosować Pydantica w pracy z modelami językowymi.

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
class User(BaseModel):
    username: str = Field(description="Username of the user")
    password: str = Field(description="Password of the user")
    is_admin: bool = Field(description="Whether or not the user is an admin")

In [ ]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": "Generate a sample user for test purposes",
        },
    ],
    text_format=User,
)

In [ ]:
response.output[0].content[0].text

`'{"username":"test_user123","password":"secureP@ssw0rd!","is_admin":false}'`

---

In [ ]:
response.output_parsed

`User(username='test_user123', password='secureP@ssw0rd!', is_admin=False)`